In [1]:
import os 

abs_path = '../'
os.chdir(abs_path) # /machine/NLP/chatbot/

from config.GlobalParams import *
from utils.Database import Database
from utils.Preprocess import Preprocess
# abs_path = './utils/'
# os.chdir(abs_path)

# 전처리 객체 생성
p = Preprocess(word2index_dic='./dict/chatbot_dict.bin',
               userdic='./dict/user_dic.tsv')

In [2]:
# 질문/답변 학습 디비 연결 객체 생성
db = Database(
    host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME
)
db.connect()    # 디비 연결
# ConnectionRefusedError: MariaDB server 실행

In [3]:
# 의도 파악]
# del IntentModel
from models.intent.IntentModel import IntentModel

intent = IntentModel(model_name='./models/intent/intent_model.h5', preprocess=p)

In [4]:
# 원문
query = "오전에 탕수육 10개 주문합니다."
predict = intent.predict_class(query) # 0: 인사, 1: 욕설, 2: 주문, 3: 예약, 4: 기타
print('predict:', predict) # predict: 2
intent_name = intent.labels[predict] # {0: "인사", 1: "욕설", 2: "주문", 3: "예약", 4: "기타"}
print('intent_name:', intent_name)

query = "안녕하세요"
predict = intent.predict_class(query)
print('predict:', predict)
intent_name = intent.labels[predict]
print('intent_name:', intent_name)

query = "짜장면 주문할게요"
predict = intent.predict_class(query)
print('predict:', predict)
intent_name = intent.labels[predict]
print('intent_name:', intent_name)

pos: [('오전', 'NNG'), ('에', 'JKB'), ('탕수육', 'NNG'), ('10', 'SN'), ('개', 'NNB'), ('주문', 'NNG'), ('하', 'XSV'), ('ㅂ니다', 'EF'), ('.', 'SF')]
keywords: ['오전', '탕수육', '10', '개', '주문']
sequences: [[269, 431, 105, 611, 3]]
padded_seqs: [[269 431 105 611   3   0   0   0   0   0   0   0   0   0   0]]
predict probability: [[8.2924382e-15 2.3796201e-05 9.9997616e-01 7.1427171e-09 1.6317978e-11]]
predict_class: tf.Tensor([2], shape=(1,), dtype=int64)
type(predict_class): <class 'tensorflow.python.framework.ops.EagerTensor'>
predict: 2
intent_name: 주문
pos: [('안녕하세요', 'NNP')]
keywords: ['안녕하세요']
sequences: [[353]]
padded_seqs: [[353   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
predict probability: [[0.6397399  0.04100596 0.06333051 0.11619129 0.13973239]]
predict_class: tf.Tensor([0], shape=(1,), dtype=int64)
type(predict_class): <class 'tensorflow.python.framework.ops.EagerTensor'>
predict: 0
intent_name: 인사
pos: [('짜장면', 'NNG'), ('주문', 'NNG'), ('하', 'XSV'), ('ㄹ게요', 'EC')]
keywords: ['짜장

In [5]:
# 개체명 인식
from models.ner.NerModel import NerModel
ner = NerModel(model_name='./models/ner/ner_model.h5', preprocess=p)

In [6]:
query = "오전에 탕수육 10개 주문합니다."
predicts = ner.predict(query)
ner_tags = ner.predict_tags(query)
print('predicts:', predicts)
print('ner_tags:', ner_tags)

predicts: [('오전', 'B_DT'), ('탕수육', 'B_FOOD'), ('10', 'O'), ('개', 'O'), ('주문', 'O')]
ner_tags: ['B_DT', 'B_FOOD']


In [7]:
print("질문 : ", query)
print("=" * 100)
print("의도 파악 : ", intent_name)
print("개체명 인식 : ", predicts)
print("답변 검색에 필요한 NER 태그 : ", ner_tags)
print("=" * 100)

질문 :  오전에 탕수육 10개 주문합니다.
의도 파악 :  주문
개체명 인식 :  [('오전', 'B_DT'), ('탕수육', 'B_FOOD'), ('10', 'O'), ('개', 'O'), ('주문', 'O')]
답변 검색에 필요한 NER 태그 :  ['B_DT', 'B_FOOD']


In [8]:
# 답변 검색
from utils.FindAnswer import FindAnswer

try:
    f = FindAnswer(db)
    answer_text, answer_image = f.search(intent_name, ner_tags)
    print('predicts:', predicts)
    print('answer_text:', answer_text)
    print('answer_image:', answer_image)
    answer = f.tag_to_word(predicts, answer_text)
except:
    answer = "죄송해요 무슨 말인지 모르겠어요"

print("답변:", answer)

db.close() # 디비 연결 끊음

predicts: [('오전', 'B_DT'), ('탕수육', 'B_FOOD'), ('10', 'O'), ('개', 'O'), ('주문', 'O')]
answer_text: {B_FOOD} 주문 처리 완료되었습니다. 
주문해주셔서 감사합니다.
answer_image: None
답변: 탕수육 주문 처리 완료되었습니다. 
주문해주셔서 감사합니다.
